In [1]:
import pandas as pd
import numpy as np

import platform
import matplotlib.pyplot as plt

# Warning 메세지 무시
import warnings
warnings.filterwarnings(module='sklearn*', action='ignore', category=DeprecationWarning)
pd.set_option('mode.chained_assignment', None) # chained_assignment 경고 무시

# 1. 졸업생 미취업위험 예측모형 개발
## 1-1) 과거(前) 졸업생 데이터 수집
* 엑셀(excel)에 저장되어 있는 예측모형구축용 과거 졸업생 원천데이터셋을 데이터프레임으로 읽어들임
* 과거 졸업생은 18, 19, 20학년도 3개년 졸업생이며, 정보공시(국세DB) 상의 취업결과정보임.
* 정보공시 취업결과는 졸업하고 2년 후에 결과가 나오기 때문에 (D-2)년 졸업생까지 예측모형구축용 데이터로 활용 가능함 (예, 22년 졸업생의 미취업위험예측을 위한 모형 구축에 20년 졸업생까지의 정보공시 취업결과데이터가 활용가능함)

In [167]:
# 예측모형구축용 과거 재학생 원천데이터셋을 데이터프레임으로 읽어들임
rawData_org = pd.read_excel('과거_졸업생_원천데이터셋.xlsx', sheet_name = '예측모형구축용')

# 과거 재학생 원천데이터셋을 저장하는 데이터프레임 확인
rawData_org

,학생ID,성별,소속학과,학점,취업준비유형,구직기초지식,구직활동준비,정보탐색활동,스펙업활동,지원활동,...,활동수준,현재준비상태,구직기술,구직서류작성능력,구직면접능력,구직의지,구직목표명확성,종합역량점수,취업유형분류,미취업여부
0,73,남자,학과08,3.25,중간집단,37.71,37.19,30.90,40.83,31.37,...,32.33,42.05,46.87,41.80,51.11,41.14,45.47,41.67,취업자(건보),0
1,229,남자,학과08,2.62,마음만 취준생,35.34,34.15,32.16,30.42,35.07,...,31.19,37.11,38.45,34.22,42.59,67.49,38.26,42.74,미취업자,1
2,541,남자,학과08,3.13,중간집단,47.16,39.83,32.16,40.83,38.78,...,35.96,43.70,43.26,34.22,52.32,56.64,50.88,47.55,취업자(건보),0
3,650,남자,학과08,3.15,중간집단,40.07,48.24,49.86,46.04,49.89,...,48.68,47.81,44.46,43.96,43.81,41.14,45.47,43.88,미취업자,1
4,1220,남자,학과08,3.84,중간집단,37.71,41.47,32.16,35.63,42.48,...,35.96,46.99,41.46,40.71,41.37,65.94,50.88,47.49,취업자(프리랜서),0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5524,5054,남자,학과06,3.31,중간집단,51.89,40.62,36.59,35.63,42.48,...,37.55,43.70,45.67,42.88,47.46,55.09,52.68,49.19,취업자(건보),0
5525,4615,남자,학과06,2.79,중간집단,40.07,43.69,44.80,40.83,46.19,...,43.68,43.70,43.26,41.80,43.81,41.14,43.67,42.37,취업자(프리랜서),0
5526,6112,남자,학과06,2.78,중간집단,49.53,40.69,34.69,40.83,38.78,...,36.87,44.52,57.09,52.61,59.62,42.69,59.89,49.98,취업자(건보),0
5527,5690,남자,학과06,3.52,중간집단,40.07,48.24,40.38,51.25,38.78,...,42.09,54.40,49.27,47.20,49.89,42.69,45.47,45.15,취업자(건보),0


## 1-2) 예측모형구축용 데이터셋 EDA 및 전처리
(1) 원천데이터셋을 구성하는 각 변수(특성)의 값 분포를 파악<br>
(2) 원천데이터셋에 존재하는 결측치(missing value) 처리<br>
(3) 원천데이터셋을 구성하는 각 변수 간의 상관 관계를 파악<br>
(4) 원천데이터셋의 변수가 너무 많을 경우, 분석차원의 축소(또는 제거)<br>
(5) 범주형 특성(categrical feature)에 대한 인코딩<br>

### (1-2-1) 원천데이터셋을 구성하는 각 변수(특성)의 값 분포 파악

In [3]:
# NULL값의 개수 구하기
rawData_org.isnull().sum()

학생ID        0
성별          0
소속학과        0
학점          0
취업준비유형      0
구직기초지식      0
구직활동준비      0
정보탐색활동      0
스펙업활동       0
지원활동        0
자격증         0
영어          0
기타어학        0
활동수준        0
현재준비상태      0
구직기술        0
구직서류작성능력    0
구직면접능력      0
구직의지        0
구직목표명확성     0
종합역량점수      0
취업유형분류      0
미취업여부       0
dtype: int64

In [4]:
# 수치형 컬럼 각각에 대한 건수, 평균, 표준편차, 최소값, 25분위값, 50분위값, 75분위값, 최대값 구하기
rawData_org[['학점','구직기초지식','구직활동준비','정보탐색활동','스펙업활동','지원활동','자격증','영어','기타어학','활동수준','현재준비상태'
            ,'구직기술','구직서류작성능력','구직면접능력','구직의지','구직목표명확성','종합역량점수']].describe()

,학점,구직기초지식,구직활동준비,정보탐색활동,스펙업활동,지원활동,자격증,영어,기타어학,활동수준,현재준비상태,구직기술,구직서류작성능력,구직면접능력,구직의지,구직목표명확성,종합역량점수
count,5529.000000,5529.000000,5529.000000,5529.000000,5529.000000,5529.000000,5529.000000,5529.000000,5529.000000,5529.000000,5529.000000,5529.000000,5529.000000,5529.000000,5529.000000,5529.000000,5529.000000
mean,3.269694,48.260228,43.555124,41.288121,42.348897,42.553851,52.566817,39.509910,27.983252,41.345927,45.807218,49.774625,48.172594,49.818624,52.438076,50.562006,48.918123
std,0.582264,9.516791,6.649745,9.207455,9.160012,9.457450,14.817237,2.142781,9.134128,9.365053,7.133739,9.738381,9.826955,9.804650,10.735773,10.270389,6.534364
min,1.250000,35.340000,12.000000,25.840000,25.210000,27.670000,32.330000,39.200000,26.150000,25.740000,29.210000,12.000000,14.750000,10.960000,13.230000,16.640000,27.630000
25%,2.880000,40.070000,38.680000,34.060000,35.630000,35.070000,39.000000,39.200000,26.150000,34.140000,39.580000,44.460000,42.880000,43.810000,44.240000,43.670000,44.250000
50%,3.300000,47.160000,42.650000,39.750000,40.830000,42.480000,50.110000,39.200000,26.150000,39.370000,44.520000,48.670000,47.200000,48.670000,51.990000,50.880000,48.400000
75%,3.700000,54.260000,47.420000,46.700000,46.040000,46.190000,61.220000,39.200000,26.150000,46.630000,50.280000,55.880000,54.780000,55.970000,59.740000,58.080000,53.250000
max,4.490000,77.900000,78.910000,90.320000,87.710000,86.930000,99.000000,65.870000,122.310000,90.240000,79.910000,80.530000,76.410000,80.300000,81.450000,74.300000,75.600000


In [5]:
# 범주형 컬럼 각각에 대한 값의 분포
# 소속학과별 건수 (건수 내림차순으로 정렬)
rawData_org[['소속학과','학생ID']].groupby('소속학과').학생ID.agg(['count']).sort_values(by=['count'], ascending=False)

,count
소속학과,
학과25,340
학과33,300
학과35,277
학과02,260
학과26,210
학과13,206
학과06,205
학과12,204
학과21,203


In [6]:
# 범주형 컬럼 각각에 대한 값의 분포
# 성별 건수 (건수 내림차순으로 정렬)
rawData_org[['성별','학생ID']].groupby('성별').학생ID.agg(['count']).sort_values(by=['count'], ascending=False)

,count
성별,
여자,3598
남자,1931


In [7]:
# 범주형 컬럼 각각에 대한 값의 분포
# 취업준비유형별 건수 (건수 내림차순으로 정렬)
rawData_org[['취업준비유형','학생ID']].groupby('취업준비유형').학생ID.agg(['count']).sort_values(by=['count'], ascending=False)

,count
취업준비유형,
중간집단,3475
취업적신호,755
준비된 취준생,681
마음만 취준생,571
준비중독,47


In [8]:
# 범주형 컬럼 각각에 대한 값의 분포
# 취업준비유형별 건수 (건수 내림차순으로 정렬)
rawData_org[['취업유형분류','학생ID']].groupby('취업유형분류').학생ID.agg(['count']).sort_values(by=['count'], ascending=False)

,count
취업유형분류,
취업자(건보),3165
미취업자,1425
제외자,529
취업자(프리랜서),225
취업자(개인창작활동종사자),95
취업자(해외취업자),72
취업자(1인창업),18


In [9]:
# 범주형 컬럼 각각에 대한 값의 분포
# 미취업여부별 건수 (건수 내림차순으로 정렬)
rawData_org[['미취업여부','학생ID']].groupby('미취업여부').학생ID.agg(['count']).sort_values(by=['count'], ascending=False)

,count
미취업여부,
0,4104
1,1425


### (1-2-2) 원천데이터셋에 존재하는 결측치(missing value) 처리
* 총 5,529건 중 결측치로 인한 제거 대상 건(row)이 없음

### (1-2-3) 원천데이터셋을 구성하는 각 변수 간의 상관 관계를 파악
* 독립변수와 종속변수(중도탈락여부)와의 상관관계 파악
 * 수치형 독립변수 : 로지스틱 회귀분석
  * 결정계수가 1에 가까울수록 계수의 절대값이 클수록 p값이 낮을수록 상관관계가 높음 (계수값이 양수이면 양의 상관관계, 음수이면 음의 상관계)
 * 범주형 독립변수 : 카이제곱 검정
  * 측정한 카이제곱통계량과 자유도를 카이제곱분포표에 적용한 결과 도출된 유의확률이 미리 설정한 유의수준(상관성의 판단기준)보다 작을 때 상관성이 있다고 판단
* 독립변수들 간의 다중공선성 파악 : 분산팽창계수
 * 다중공선성 : 하나의 독립변수가 다른 여러 개의 독립변수들로 잘 예측되는 성질
  * 분산팽창계수(VIF, Variance Inflation Factor)다)가 보통 10보다 크면 다중공선성이 있다고 판단 -> 다중공선성이 있는 독립변수는 예측모형의 최종 독립변수에서 제외

In [10]:
# 수치형 독립변수와 종속변수(중도탈락여부)와의 상관성 파악을 위한 로지스틱 회귀분석(Logistic Regression)
# features : 독립변수 데이터프레임
# dropout : 종속변수 리스트
features = rawData_org[['학점','구직기초지식','구직활동준비','정보탐색활동','스펙업활동','지원활동','자격증','영어','기타어학','활동수준','현재준비상태'
            ,'구직기술','구직서류작성능력','구직면접능력','구직의지','구직목표명확성','종합역량점수']]
dropout = rawData_org['미취업여부']

In [11]:
# train/test set 분리하기
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(features, dropout)

In [12]:
# 데이터 정규화(표준화) : 각 독립변수의 scale이 달라서 생길 수 있는 종속변수에 끼칠 영향도의 차이를 없애기 위해 동일 scale로 표준화 진행
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train_std = scaler.fit_transform(X_train)
X_test_std = scaler.fit_transform(X_test)

In [13]:
# 로지스틱 분석 시행
import statsmodels.api as sm

logreg = sm.Logit(y_train, sm.add_constant(X_train_std)).fit()
logreg.summary()

Optimization terminated successfully.
         Current function value: 0.555939
         Iterations 7


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                  미취업여부   No. Observations:                 4146
Model:                          Logit   Df Residuals:                     4128
Method:                           MLE   Df Model:                           17
Date:                Sat, 20 Jul 2024   Pseudo R-squ.:                 0.03074
Time:                        06:02:52   Log-Likelihood:                -2304.9
converged:                       True   LL-Null:                       -2378.0
Covariance Type:            nonrobust   LLR p-value:                 1.351e-22
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -1.0870      0.037    -29.592      0.000      -1.159      -1.015
x1            -0.3004      0.051     -5.857      0.000      -0.401      -0.200
x2            32.4152     22.292      1.454      0.146     -11.277      76.107
x3            22.7916     15.635      1.458      0.145      -7.853      53.436
x4             0.0990      0.389      0.255      0.799      -0.663       0.861
x5            -0.0326      0.330     -0.099      0.921      -0.679       0.614
x6            -0.0032      0.476     -0.007      0.995      -0.937       0.931
x7            -0.1737      0.093     -1.876      0.061      -0.355       0.008
x8             0.0491      0.037      1.316      0.188      -0.024       0.122
x9             0.0444      0.040      1.107      0.268      -0.034       0.123
x10           -0.1310      0.605     -0.217      0.829      -1.317       1.055
x11            0.0215      0.538      0.040      0.968      -1.032       1.075
x12          182.1189     95.791      1.901      0.057      -5.629     369.867
x13          -83.4417     54.065     -1.543      0.123    -189.408      22.524
x14          -74.1861     48.026     -1.545      0.122    -168.315      19.942
x15           36.0653     24.896      1.449      0.147     -12.730      84.861
x16           34.8431     24.077      1.447      0.148     -12.347      82.033
x17         -111.0172     76.363     -1.454      0.146    -260.686      38.651
==============================================================================
"""

* '학점'(x1),'구직기초지식'(x2),'구직활동준비'(x3),'정보탐색활동'(x4),'스펙업활동'(x5),'지원활동'(x6),'자격증'(x7),'영어'(x8),'기타어학'(x9),'활동수준'(x10),'현재준비상태'(x11),'구직기술'(x12),'구직서류작성능력'(x13),'구직면접능력'(x14),'구직의지'(x15),'구직목표명확성'(x16),'종합역량점수'(x17)
* (유사)결정계수(Pseudo R-squ.) 값으로 미루어 모델의 설명력은 약한 편임
* coef는 각 독립변수의 계수로 '학점'(x1),'자격증'(x7),'활동수준'(x10),'구직서류작성능력'(x13),'구직면접능력'(x14),'종합역량점수'(x17)의 값이 클수록 취업에 성공하는 경향성을 띰
* P-value 관점에서는 '학점'(x1),'자격증'(x7),'구직기술'(x12) 외에는 상대적으로 설명력이 크게 약함
* 로지스틱 분석 통계 상으로는 신뢰도 높은 강건한 모델로 보기는 어려움

In [14]:
# 로지스틱 회귀모델 생성 및 평가
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
model.fit(X_train_std, y_train)

LogisticRegression()

In [15]:
print(model.score(X_train_std, y_train))
print(model.score(X_test_std, y_test))

0.7390255668113844
0.7483731019522777


In [16]:
# 범주형 독립변수와 종속변수(미취업여부)와의 상관성 파악을 위한 카이제곱 검정(Chi-square Validation)
# 카이제곱검정을 위한 패키지 임포트
from scipy.stats import chi2_contingency

# 소속학과와 미취업여부 상관성 파악
# 검증함수 활용을 위한 데이터셋 재구조화
features = rawData_org[['소속학과','미취업여부','학생ID']]
cnt_df_1 = features.loc[features['미취업여부']==1].groupby(['소속학과','미취업여부']).학생ID.agg(['count'])
cnt_df_0 = features.loc[features['미취업여부']==0].groupby(['소속학과','미취업여부']).학생ID.agg(['count'])
cnt_df = pd.merge(cnt_df_1,cnt_df_0, how='inner',on='소속학과')
cnt_df.rename(columns={'count_x':'미취업','count_y':'취업'}, inplace=True)

# 검증함수 수행
chiresult = chi2_contingency(cnt_df, correction=False)
print('(소속학과, 미취업여부)')
print('Chi square: {}'.format(chiresult[0]))
print('p-value: {}'.format(chiresult[1]))
if chiresult[1] <= 0.05 :  # 유의수준 설정은 사용자정의인데, 대체로 0.05로 설정
    print(' -> 유의확률(p-value)이 유의수준(0.05) 이하이므로 소속학과는 미취업여부와 상관성이 있다')
else :
    print(' -> 유의확률(p-value)이 유의수준(0.05) 초과이므로 소속학과는 미취업여부와 상관성이 없다(독립변수에서 제외 검토)')

(소속학과, 미취업여부)
Chi square: 251.85676297815456
p-value: 6.637873053774246e-37
 -> 유의확률(p-value)이 유의수준(0.05) 이하이므로 소속학과는 미취업여부와 상관성이 있다


In [17]:
# 범주형 독립변수와 종속변수(미취업여부)와의 상관성 파악을 위한 카이제곱 검정(Chi-square Validation)
# 카이제곱검정을 위한 패키지 임포트
from scipy.stats import chi2_contingency

# 성별과 미취업여부 상관성 파악
# 검증함수 활용을 위한 데이터셋 재구조화
features = rawData_org[['성별','미취업여부','학생ID']]
cnt_df_1 = features.loc[features['미취업여부']==1].groupby(['성별','미취업여부']).학생ID.agg(['count'])
cnt_df_0 = features.loc[features['미취업여부']==0].groupby(['성별','미취업여부']).학생ID.agg(['count'])
cnt_df = pd.merge(cnt_df_1,cnt_df_0, how='inner',on='성별')
cnt_df.rename(columns={'count_x':'미취업','count_y':'취업'}, inplace=True)

# 검증함수 수행
chiresult = chi2_contingency(cnt_df, correction=False)
print('(성별, 미취업여부)')
print('Chi square: {}'.format(chiresult[0]))
print('p-value: {}'.format(chiresult[1]))
if chiresult[1] <= 0.05 :  # 유의수준 설정은 사용자정의인데, 대체로 0.05로 설정
    print(' -> 유의확률(p-value)이 유의수준(0.05) 이하이므로 성별과 미취업여부와 상관성이 있다')
else :
    print(' -> 유의확률(p-value)이 유의수준(0.05) 초과이므로 성별과 미취업여부와 상관성이 없다(독립변수에서 제외 검토)')

(성별, 미취업여부)
Chi square: 0.5675321538703202
p-value: 0.45124124251189934
 -> 유의확률(p-value)이 유의수준(0.05) 초과이므로 성별과 미취업여부와 상관성이 없다(독립변수에서 제외 검토)


In [18]:
# 범주형 독립변수와 종속변수(미취업여부)와의 상관성 파악을 위한 카이제곱 검정(Chi-square Validation)
# 카이제곱검정을 위한 패키지 임포트
from scipy.stats import chi2_contingency

# 취업준비유형과 미취업여부 상관성 파악
# 검증함수 활용을 위한 데이터셋 재구조화
features = rawData_org[['취업준비유형','미취업여부','학생ID']]
cnt_df_1 = features.loc[features['미취업여부']==1].groupby(['취업준비유형','미취업여부']).학생ID.agg(['count'])
cnt_df_0 = features.loc[features['미취업여부']==0].groupby(['취업준비유형','미취업여부']).학생ID.agg(['count'])
cnt_df = pd.merge(cnt_df_1,cnt_df_0, how='inner',on='취업준비유형')
cnt_df.rename(columns={'count_x':'미취업','count_y':'취업'}, inplace=True)

# 검증함수 수행
chiresult = chi2_contingency(cnt_df, correction=False)
print('(취업준비유형, 미취업여부)')
print('Chi square: {}'.format(chiresult[0]))
print('p-value: {}'.format(chiresult[1]))
if chiresult[1] <= 0.05 :  # 유의수준 설정은 사용자정의인데, 대체로 0.05로 설정
    print(' -> 유의확률(p-value)이 유의수준(0.05) 이하이므로 성별과 미취업여부와 상관성이 있다')
else :
    print(' -> 유의확률(p-value)이 유의수준(0.05) 초과이므로 성별과 미취업여부와 상관성이 없다(독립변수에서 제외 검토)')

(취업준비유형, 미취업여부)
Chi square: 56.234172574626335
p-value: 1.7908222915589357e-11
 -> 유의확률(p-value)이 유의수준(0.05) 이하이므로 성별과 미취업여부와 상관성이 있다


* '취업유형분류' 특성은 '미취업여부' 특성과 마찬가지로 취업결과 특성 중 하나이며, 종속변수(미취업여부)와 인과관계가 있는 독립변수로 보기 어려우므로 상관분석에서 제외함

In [19]:
# 분산팽창계수를 구하기 위한 패키지(모듈) 임포트
from statsmodels.stats.outliers_influence import variance_inflation_factor

# 다중공선성 파악을 위한 분산팽창계수 구하기
data = rawData_org[['학점','구직기초지식','구직활동준비','정보탐색활동','스펙업활동','지원활동','자격증','영어','기타어학','활동수준','현재준비상태'
            ,'구직기술','구직서류작성능력','구직면접능력','구직의지','구직목표명확성','종합역량점수']]

# 데이터 정규화(표준화) : 각 독립변수의 scale이 달라서 생길 수 있는 종속변수에 끼칠 영향도의 차이를 없애기 위해 동일 scale로 표준화 진행
from sklearn.preprocessing import StandardScaler

# 정규화(표준화) 수행
scaler = StandardScaler()
data_std = scaler.fit_transform(data)

# VIF 출력을 위한 데이터 프레임 형성
vif = pd.DataFrame()

# VIF 값과 각 Feature 이름에 대해 설정
vif["VIF Factor"] = [variance_inflation_factor(data_std, i) for i in range(data_std.shape[1])]
vif["features"] = data.columns

# VIF 값이 높은 순으로 정렬
vif = vif.sort_values(by="VIF Factor", ascending=False)
vif = vif.reset_index().drop(columns='index')
vif

,VIF Factor,features
0,7.122626e+06,구직기술
1,4.464314e+06,종합역량점수
2,2.268370e+06,구직서류작성능력
3,1.784937e+06,구직면접능력
4,4.821496e+05,구직의지
5,4.411130e+05,구직목표명확성
6,3.787368e+05,구직기초지식
7,1.853201e+05,구직활동준비
8,1.926865e+02,활동수준
9,1.424456e+02,현재준비상태


* 각 독립변수에 대해서 분산팽창계수가 10 미만이므로 모두 유의미한 독립변수로 간주할 수 있음

### (1-2-4) 분석차원의 축소(또는 제거)
* 특성 선택(feature selection) : 여러 특성(변수) 중에서 종속변수에 영향도가 높은 일부 특성군을 선택하여 최종 독립변수로 활용
 * 로지스틱 회귀분석 결과 p값이 작고 계수(coef)의 절대값이 큰 변수 위주로 선택 가능
  * 본 예제에서 학점'(x1),'자격증'(x7) 등이 우선 고려 대상
* 특성 추출(feature extraction) : 여러 특성(변수)를 합성하여 저차원의 새로운 특성을 추출해서 최종 독립변수로 활용
  * 대표적으로 주성분분석(PCA) 기법이 있음

In [20]:
# 주성분분석
# 주성분분석을 위한 수치형 독립변수 선택 (나이를 제외하고 교과, 비교과, 장학, 상담과 관련 있는 변수 선택)
data = rawData_org[['학점','구직기초지식','구직활동준비','정보탐색활동','스펙업활동','지원활동','자격증','영어','기타어학','활동수준','현재준비상태'
            ,'구직기술','구직서류작성능력','구직면접능력','구직의지','구직목표명확성','종합역량점수']]

# 데이터 정규화(표준화) : 각 독립변수의 scale이 달라서 생길 수 있는 종속변수에 끼칠 영향도의 차이를 없애기 위해 동일 scale로 표준화 진행
from sklearn.preprocessing import StandardScaler

# 정규화(표준화) 수행
scaler = StandardScaler()
data_std = scaler.fit_transform(data)

features = ['학점','구직기초지식','구직활동준비','정보탐색활동','스펙업활동','지원활동','자격증','영어','기타어학','활동수준','현재준비상태'
            ,'구직기술','구직서류작성능력','구직면접능력','구직의지','구직목표명확성','종합역량점수']
pd.DataFrame(data_std, columns=features)

# 주성분분석을 위한 패키지(모듈) 임포트
from sklearn.decomposition import PCA

# 주성분 모형변수
pca = PCA(n_components=9)

# 원속성을 주성분으로 변환
printcipalComponents = pca.fit_transform(data_std)
principalDf = pd.DataFrame(data=printcipalComponents, columns = ['1', '2', '3','4','5','6','7','8','9'])

# 각 주성분의 설명력 출력 (설명력의 누적값을 활용하여 주성분의 갯수 결정. 대체로 90~95% 이상에서 결정)
print(pca.explained_variance_ratio_)

[0.43315788 0.11777677 0.10549853 0.07035851 0.06574896 0.05319478
 0.04486598 0.03890175 0.02814036]


* 설명력(설명가능한 분산량)이 0.95 이상이 되려면 9개의 주성분(PC) 필요
 * 0.43315788 + 0.11777677 + 0.10549853 + 0.07035851 + 0.06574896 + 0.05319478 + 0.04486598 + 0.03890175 + 0.02814036 = 0.95764352

In [21]:
# 주성분을 9개로 결정 (설명력 : 95.8%)
pca = PCA(n_components=9)

printcipalComponents = pca.fit_transform(data_std)
principalDf = pd.DataFrame(data=printcipalComponents, columns = ['PC1', 'PC2', 'PC3', 'PC4', 'PC5', 'PC6', 'PC7', 'PC8', 'PC9'])
principalDf

,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9
0,-2.590878,-0.069594,-0.504216,0.002610,0.994219,-0.184698,-0.098340,0.162975,-0.585677
1,-3.876511,-0.785549,0.501585,0.199914,-1.352149,0.482042,-0.868625,0.922238,0.507174
2,-1.504776,-0.139292,-0.100897,-0.181941,-0.846980,0.120900,0.089134,-0.006653,-0.026140
3,0.085842,1.778657,1.132428,-0.550336,0.957298,0.373262,-0.304712,0.435523,-0.553257
4,-1.615932,0.340460,-0.612679,-0.395588,-1.518412,-1.055718,-0.646428,0.366388,-0.065109
...,...,...,...,...,...,...,...,...,...
5524,-0.792705,-0.161400,-0.319222,-0.301334,-0.698196,0.191210,0.504455,-0.358300,0.194465
5525,-1.175531,1.041130,1.115985,-0.246887,0.868236,0.779089,-0.117448,0.358714,-0.449510
5526,0.046646,-1.276489,-0.823675,0.018659,0.896394,0.788150,0.666330,-0.282521,-0.904713
5527,0.106976,1.631920,-0.629314,-0.682998,1.193236,-0.113380,-0.471451,0.527172,-0.436778


* 17개의 원속성이 9개의 주성분으로 변환되어 분석차원 축소효과를 거둠 (설명력 : 약 95.8%)

In [22]:
# 각 주성분(PC) 값을 결정하는데 있어서 원속성들의 영향력 파악 (주성분의 특성 파악을 위해 필요)
pca_df = pd.DataFrame(pca.components_, columns=['학점','구직기초지식','구직활동준비','정보탐색활동','스펙업활동','지원활동','자격증','영어','기타어학','활동수준','현재준비상태'
            ,'구직기술','구직서류작성능력','구직면접능력','구직의지','구직목표명확성','종합역량점수'])
pca_df.index = ['PC1','PC2','PC3','PC4','PC5','PC6','PC7','PC8','PC9']
pca_df

,학점,구직기초지식,구직활동준비,정보탐색활동,스펙업활동,지원활동,자격증,영어,기타어학,활동수준,현재준비상태,구직기술,구직서류작성능력,구직면접능력,구직의지,구직목표명확성,종합역량점수
PC1,0.092257,0.244593,0.328915,0.297989,0.269908,0.303599,0.167661,0.070660,0.070305,0.317361,0.195342,0.277245,0.266479,0.256867,0.113002,0.236136,0.332188
PC2,0.234454,-0.048991,0.301007,0.104670,0.183306,0.076190,0.299936,0.181292,0.184857,0.122737,0.394998,-0.355763,-0.319422,-0.354995,-0.231000,-0.172603,-0.189219
PC3,-0.273151,0.063541,0.002741,0.328918,0.311730,0.314111,-0.392124,-0.080369,-0.112716,0.345413,-0.449427,-0.180720,-0.175896,-0.164946,-0.086322,-0.120850,-0.101157
PC4,-0.104495,0.020243,-0.067089,-0.017714,-0.062519,-0.007076,-0.305825,0.661778,0.645161,-0.027571,-0.087545,0.076303,0.060107,0.085604,-0.073561,-0.034448,-0.020021
PC5,-0.014954,-0.121906,0.035778,0.016282,0.044599,0.033057,0.055743,-0.085001,-0.080130,0.033772,0.022325,0.305659,0.306863,0.268486,-0.707756,-0.350633,-0.279923
PC6,-0.841662,0.135332,0.029102,0.034136,-0.143487,0.017630,0.444941,-0.094411,0.175045,-0.023767,0.081573,-0.033831,-0.045709,-0.016456,-0.011453,-0.040591,0.018723
PC7,-0.028346,0.533039,-0.064504,-0.043467,-0.160633,-0.020907,0.026555,0.447537,-0.494427,-0.071879,-0.024913,-0.082815,-0.052876,-0.106836,-0.367396,0.263031,0.079390
PC8,-0.182448,-0.478932,0.056764,0.047337,0.125871,-0.005064,0.137543,0.539265,-0.477770,0.051549,0.036401,0.072025,0.050552,0.087793,0.324515,-0.220236,-0.069053
PC9,0.127824,0.531413,-0.005274,-0.004538,-0.124134,0.030713,-0.006092,0.008497,-0.034338,-0.024018,0.022844,0.035578,0.029544,0.038196,0.310846,-0.762583,0.026760


* PC1은 종합역량점수를 상대적으로 많이 반영
* PC2는 구직활동준비, 현재준비상태, 구직기술, 구직서류작성능력, 구직면접능력 등 구직 준비나 스킬을 상대적으로 많이 반영함
* PC3은 정보탐색활동, 스펙업활동, 지원활동, 자격증, 활동수준, 현재준비상태 등 활동이나 역량을 상대적으로 많이 반영함
* PC4는 영어, 기타어학 등 어학역량을 상대적으로 많이 반영함
* PC5는 구직의지나 구직목표명확성을 상대적으로 많이 반영함
* PC6는 학점과 자격증을 상대적으로 많이 반영함
* PC7, PC8은 구직기초지식, 영어, 기타어학, 구직의지 등 역량과 의지를 상대적으로 많이 반영함
* PC9는 구직기초지식, 구직의지, 구직목표명확성을 상대적으로 많이 반영함

### (1-2-5) 범주형 특성(categrical feature)에 대한 인코딩
* 범주형 특성에 대해서 예측모형구축용과 예측용 데이터셋에서 동일한 범주값이 동일하게 인코딩됨을 보장하기 위해서 예측용 데이터셋도 미리 읽어들여 NULL값을 가진 건은 삭제하고 난 다음 예측모형구축용과 예측용 데이터셋을 한꺼번에 인코딩함

In [168]:
# 예측용 현 입학초신입생 원천데이터셋을 데이터프레임으로 읽어들임
# rawData_org : 예측모형구축용 원천데이터셋, rawData2_org : 예측용 원천데이터셋
rawData2_org = pd.read_excel('현재_졸업(예정)자_원천데이터셋.xlsx', sheet_name = '예측용(22년졸업)')
rawData3_org = pd.read_excel('현재_졸업(예정)자_원천데이터셋.xlsx', sheet_name = '예측용(23년졸업예정)')

rawData_org['구분'] = '예측모형구축용'
rawData2_org['구분'] = '예측용(졸업생)'
rawData3_org['구분'] = '예측용(졸업예정자)'

rawData_all = pd.concat([rawData_org, rawData2_org, rawData3_org])

rawData_all = rawData_all[['학생ID','소속학과','성별','취업준비유형','학점','구직기초지식','구직활동준비','정보탐색활동','스펙업활동','지원활동','자격증','영어','기타어학','활동수준','현재준비상태'
                   ,'구직기술','구직서류작성능력','구직면접능력','구직의지','구직목표명확성','미취업여부','구분']]

# 범주형 특성에 대한 레이블 인코딩(label encoding)
categorical_columns = ['소속학과', '성별', '취업준비유형', '미취업여부']

In [169]:
# 레이블인코딩을 위한 패키지(모듈) 임포트
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
for feature in categorical_columns:
    rawData_all[feature] = le.fit_transform(rawData_all[feature])

# rawData_e : 전처리된 예측모형구축용 데이터셋, rawData2_e : 전처리된 예측용 데이터셋
rawData_e = rawData_all[rawData_all['구분'] == '예측모형구축용'].iloc[:, :-1]
rawData2_e = rawData_all[rawData_all['구분'] == '예측용(졸업생)'].iloc[:, :-1]
rawData3_e = rawData_all[rawData_all['구분'] == '예측용(졸업예정자)'].iloc[:, :-1]

## 1-3) 예측모형 설계·구축
(1) 독립변수와 종속변수의 확정<br>
(2) 분석대상데이터셋을 트레이닝셋과 테스트셋으로 분리<br>
(3) 종속변수인 중도탈락여부 클래스의 분포 불균형 해소<br>
(4) 트레이닝셋을 대상으로 분류예측알고리듬 별로 학습을 수행하고 성능 측정/비교하여 가장 우수한 알고리듬 선택<br>
(5) 독립변수 별 중도탈락예측 영향도 파악<br>

### (1-3-1) 독립변수와 종속변수의 확정
* 독립변수의 개수(차원수)가 지나치게 많다면 예측모형의 성능이 떨어지고 과적합(Overfitting)의 문제가 생길 수 있음
* -> 로지스틱회귀, 카이제곱검정, 공선성분석 등을 통해 독립적이면서 종속변수에 영향도가 높은 변수들을 선택하거나 주성분분석과 같이 차원축소 방식을 통해 독립변수의 개수를 줄일 필요 있음

In [222]:
# 독립변수+종속변수 확정
# 독립변수 : '소속학과','성별','취업준비유형','학점','구직기초지식','구직활동준비','정보탐색활동','스펙업활동','지원활동','자격증','영어','기타어학','활동수준','현재준비상태'
#           ,'구직기술','구직서류작성능력','구직면접능력','구직의지','구직목표명확성' ('종합역량점수'의 경우, 취업준비도검사결과 종합으로 각 항목값으로 이미 특성이 반영된 것으로 판단하여 제외)
# 종속변수 : 미취업여부

# feature_set_shrink : EDA를 통해 유의미성이 낮거나 종속변수와의 상관성이 낮은 독립변수를 제거할 것인지 여부 (1:제거, 0:제거하지않음)
# 유의미성이 낮은 변수 : 영어, 기타어학
# 상관성이 낮은 변수 : 성별
feature_set_shrink = 0

if feature_set_shrink == 1:
    feature_list = ['소속학과','취업준비유형','학점','구직기초지식','구직활동준비','정보탐색활동','스펙업활동','지원활동','자격증','활동수준','현재준비상태'
                   ,'구직기술','구직서류작성능력','구직면접능력','구직의지','구직목표명확성','미취업여부']
else:
    feature_list = ['소속학과','성별','취업준비유형','학점','구직기초지식','구직활동준비','정보탐색활동','스펙업활동','지원활동','자격증','영어','기타어학','활동수준','현재준비상태'
                   ,'구직기술','구직서류작성능력','구직면접능력','구직의지','구직목표명확성','미취업여부']

analData = rawData_e[feature_list]
analData

,소속학과,취업준비유형,학점,구직기초지식,구직활동준비,정보탐색활동,스펙업활동,지원활동,자격증,활동수준,현재준비상태,구직기술,구직서류작성능력,구직면접능력,구직의지,구직목표명확성,미취업여부
0,7,3,3.25,37.71,37.19,30.90,40.83,31.37,52.33,32.33,42.05,46.87,41.80,51.11,41.14,45.47,0
1,7,0,2.62,35.34,34.15,32.16,30.42,35.07,39.00,31.19,37.11,38.45,34.22,42.59,67.49,38.26,1
2,7,3,3.13,47.16,39.83,32.16,40.83,38.78,50.11,35.96,43.70,43.26,34.22,52.32,56.64,50.88,0
3,7,3,3.15,40.07,48.24,49.86,46.04,49.89,61.22,48.68,47.81,44.46,43.96,43.81,41.14,45.47,1
4,7,3,3.84,37.71,41.47,32.16,35.63,42.48,45.67,35.96,46.99,41.46,40.71,41.37,65.94,50.88,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5524,5,3,3.31,51.89,40.62,36.59,35.63,42.48,56.78,37.55,43.70,45.67,42.88,47.46,55.09,52.68,0
5525,5,3,2.79,40.07,43.69,44.80,40.83,46.19,56.78,43.68,43.70,43.26,41.80,43.81,41.14,43.67,0
5526,5,3,2.78,49.53,40.69,34.69,40.83,38.78,59.00,36.87,44.52,57.09,52.61,59.62,42.69,59.89,0
5527,5,3,3.52,40.07,48.24,40.38,51.25,38.78,65.67,42.09,54.40,49.27,47.20,49.89,42.69,45.47,0


### (1-3-2) 분석대상데이터셋을 트레이닝셋과 테스트셋으로 분리
* 전처리된 데이터셋을 독립변수 데이터셋과 종속변수 데이터셋으로 분리
* 트레이닝셋과 테스트셋 비율을 7:3으로 하여 분석대상데이터셋 분리

In [223]:
# 예측모형구축
# 소스 데이터프레임에서 분류(classification)을 위한 속성 집합
X = analData.loc[:, feature_list[:-1]]  # 독립변수 데이터셋
y = analData.loc[:, '미취업여부']  # 종속변수 데이터셋

# 트레이닝셋과 테스트셋 분리에 관련된 모듈 임포트
from sklearn.model_selection import train_test_split

# 자동으로 데이터셋을 트레이닝셋과 테스트셋으로 분리해주는 함수로
# 트레이닝셋과 데이터셋의 비율을 7:3으로 세팅함
# X_train : 트레이닝셋의 독립변수 데이터셋, y_train : 트레이닝셋의 종속변수 데이터셋
# X_test : 테스트셋의 독립변수 데이터셋, y_test : 테스트셋의 종속변수 데이터셋
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

### (1-3-3) 종속변수인 중도탈락여부 클래스의 분포 불균형 해소
* 미취업위험여부 변수값은 미취업(1)과 취업(0) 등 2가지 종류가 있는데, ‘학업유지’ 값을 갖는 경우가 ‘중도탈락’ 값을 갖는 경우보다 월등히 많아 분류성능지표(정확도 등)를 왜곡할 수 있음
* 이를 해결하기 위해서 적은 분포를 갖는 클래스의 투플을 인공적으로 만들어 클래스 분포의 균형을 이루는 오버샘플링(oversampling) 기법을 적용함. 대표적으로 SMOTE(Synthetic Minority Oversampling Technique) 기법이 있음
 * SMOTE(Synthetic Minority Oversampling Technique) : 데이터의 개수가 적은 클래스의 표본을 가져온 뒤 임의의 값을 추가하여 새로운 샘플을 만들어 데이터에 추가하는 오버샘플링 방식

In [224]:
# SMOTE기법(Over-sampling) 적용을 위한 패키지(모듈) 임포트
from imblearn.over_sampling import SMOTE

# 트레이닝셋에 대한 SMOTE 적용
smote = SMOTE(random_state=0)
X_train_over,y_train_over = smote.fit_resample(X_train,y_train)

print('SMOTE 적용 전 트레이닝셋의 행열 갯수: ', X_train.shape, y_train.shape)
print('SMOTE 적용 후 트레이닝셋의 행열 갯수: ', X_train_over.shape, y_train_over.shape)
print('SMOTE 적용 후 트레이닝셋의 종속변수 값 분포: \n', pd.Series(y_train_over).value_counts())

# 테스트셋에 대한 SMOTE 적용
X_test_over,y_test_over = smote.fit_resample(X_test,y_test)
print('SMOTE 적용 전 테스트셋의 행열 갯수: ', X_test.shape, y_test.shape)
print('SMOTE 적용 후 테스트셋의 행열 갯수: ', X_test_over.shape, y_test_over.shape)
print('SMOTE 적용 후 테스트셋의 종속변수 값 분포: \n', pd.Series(y_test_over).value_counts())

SMOTE 적용 전 트레이닝셋의 행열 갯수:  (3870, 16) (3870,)
SMOTE 적용 후 트레이닝셋의 행열 갯수:  (5772, 16) (5772,)
SMOTE 적용 후 트레이닝셋의 종속변수 값 분포: 
 미취업여부
0    2886
1    2886
Name: count, dtype: int64
SMOTE 적용 전 테스트셋의 행열 갯수:  (1659, 16) (1659,)
SMOTE 적용 후 테스트셋의 행열 갯수:  (2436, 16) (2436,)
SMOTE 적용 후 테스트셋의 종속변수 값 분포: 
 미취업여부
0    1218
1    1218
Name: count, dtype: int64


* SMOTE 적용 후 트레이닝셋의 1(중도탈락)군과 0(학업유지)군의 건수 분포는 각각 2,886건으로 동일함
* SMOTE 적용 후 테스트셋의 1(중도탈락)군과 0(학업유지)군의 건수 분포는 각각 1,218건으로 동일함

### (1-3-4) 트레이닝셋을 대상으로 분류예측알고리듬 별로 학습을 수행하고 성능 측정/비교하여 가장 우수한 알고리듬 선택
* 비교 대상 분류예측알고리듬 : 의사결정트리(decision tree), 랜덤포레스트(random forest), XGBoost, LightGBM

In [225]:
# 분류예측 성능측정을 위한 패키지(모듈) 임포트
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# 트레이닝셋 학습 및 테스트셋 검증
def f_model_fit(cls, model, X_train, X_test, y_train, y_test):
    model.fit(X_train, y_train)   # 트레이닝셋 학습
    pred = model.predict(X_test)  # 테스트셋 검증
    f_metrics(cls, y_test, pred)  # 성능 측정

# 분류예측 성능측정
def f_metrics(cls, y_test, pred):
    accuracy = accuracy_score(y_test, pred)    # 정확도(accuracy) 측정
    precision = precision_score(y_test, pred)  # 정밀도(precision) 측정
    recall = recall_score(y_test, pred)        # 재현율(recall) 측정
    f1 = f1_score(y_test, pred)                # f1-score 측정
    auc = roc_auc_score(y_test, pred)          # auc 측정
    print(cls + '정확도 : {0:.3f}, 정밀도 : {1:.3f}, 재현율 : {2:.3f}, f1-score : {3:.3f}, AUC : {4:.3f}'.format(accuracy, precision, recall, f1, auc))

# 분석기법 : 의사결정트리(Decision_tree)
from sklearn.tree import DecisionTreeClassifier   # 의사결정트리 기법에 관련된 모듈 임포트

# DecisionTreeClassifier() : 의사결정트리 모형설정 함수
# 하이퍼패러미터(hyper parameter)
# criterion : 노드 분할을 위한 품질측정방법 (gini(default), log_loss, entropy 중 택1)
# max_depth : 트리의 최대 깊이 (default : None)
# random_state : 의사결정트리 생성 시 활용하는 데이터 패턴 지정. None(default)이면 트리를 만들 때마다 활용 데이터패턴이 달라져
# 결과가 달라지며, 특정 숫자를 준다면 활용 데이터패턴이 일정하여 동일한 결과를 냄
decision_tree = DecisionTreeClassifier(criterion='entropy', max_depth=4, random_state=0)
f_model_fit('Decision_tree(SMOTE적용 전) -> ', decision_tree, X_train, X_test, y_train, y_test)
f_model_fit('Decision_tree(SMOTE적용 후) -> ', decision_tree, X_train_over, X_test_over, y_train_over, y_test_over)
print('\n')

# 분석기법 : 랜덤포레스트(Random_forest Classification)
from sklearn.ensemble import RandomForestClassifier  # 랜덤포레스트 기법에 관련된 모듈

# RandomForestClassifier() : 랜덤포레스트 모형설정 함수
# 하이퍼패러미터(hyper parameter)
# n_estimators : 모델에서 사용할 트리 갯수(학습시 생성할 트리 갯수)
# random_state : 의사결정트리 생성 시 활용하는 데이터 패턴 지정. None(default)이면 트리를 만들 때마다 활용 데이터패턴이 달라져
# 결과가 달라지며, 특정 숫자를 준다면 활용 데이터패턴이 일정하여 동일한 결과를 냄
random_forest = RandomForestClassifier(n_estimators=20, random_state=0)
f_model_fit('Random_forest(SMOTE적용 전) -> ', random_forest, X_train, X_test, y_train, y_test)
f_model_fit('Random_forest(SMOTE적용 후) -> ', random_forest, X_train_over, X_test_over, y_train_over, y_test_over)
print('\n')

# 분석기법 : XGBoost
import xgboost as xgb # XGBoost 기법에 관련된 모듈

# XGBClassifier() : XGBoost 모형설정 함수
# 하이퍼패러미터(hyper parameter)
# max_depth : 트리의 최대 깊이(default:6)
# n_estimators : 모델에서 사용할 트리 갯수(학습시 생성할 트리 갯수)
# learning_rate : 학습 단계별로 이전의 결과 가중치를 얼만큼 사용할지 결정
# eval_metric : 오차 계산 방식 (rmse, mae, logloss, error, merror, mlogloss, auc)
gbm = xgb.XGBClassifier(max_depth=4, n_estimators=500, learning_rate=0.05, eval_metric='logloss', use_label_encoder=False)
f_model_fit('XGBoost(SMOTE적용 전) -> ', gbm, X_train, X_test, y_train, y_test)
f_model_fit('XGBoost(SMOTE적용 후) -> ', gbm, X_train_over, X_test_over, y_train_over, y_test_over)
print('\n')

# 분석기법 : LightGBM
from lightgbm import LGBMClassifier  # LightGBM 기법에 관련된 모듈

# LGBMClassifier() : LightGBM 모형설정 함수
# 하이퍼패러미터(hyper parameter)
# n_estimators : 모델에서 사용할 트리 갯수(학습시 생성할 트리 갯수)
# n_jobs : 학습을 위해 사용할 스레드 개수(-1은 모든 스레드를 사용함을 의미)
# boost_from_average : 불균형한 데이터 세트에서 예측 성능이 매우 저조할 경우 False로 설정 필요
lgb = LGBMClassifier(n_estimators=1000,num_leaves=64,n_jobs=-1,boost_from_average=False,force_col_wise=True)
f_model_fit('LightGBM(SMOTE적용 전) -> ', lgb, X_train, X_test, y_train, y_test)
f_model_fit('LightGBM(SMOTE적용 후) -> ', lgb, X_train_over, X_test_over, y_train_over, y_test_over)

Decision_tree(SMOTE적용 전) -> 정확도 : 0.728, 정밀도 : 0.407, 재현율 : 0.050, f1-score : 0.089, AUC : 0.512
Decision_tree(SMOTE적용 후) -> 정확도 : 0.585, 정밀도 : 0.578, 재현율 : 0.631, f1-score : 0.604, AUC : 0.585


Random_forest(SMOTE적용 전) -> 정확도 : 0.722, 정밀도 : 0.391, 재현율 : 0.082, f1-score : 0.135, AUC : 0.518
Random_forest(SMOTE적용 후) -> 정확도 : 0.651, 정밀도 : 0.780, 재현율 : 0.422, f1-score : 0.548, AUC : 0.651


XGBoost(SMOTE적용 전) -> 정확도 : 0.723, 정밀도 : 0.427, 재현율 : 0.120, f1-score : 0.188, AUC : 0.531
XGBoost(SMOTE적용 후) -> 정확도 : 0.782, 정밀도 : 0.877, 재현율 : 0.657, f1-score : 0.751, AUC : 0.782


[LightGBM] [Info] Number of positive: 984, number of negative: 2886
[LightGBM] [Info] Total Bins 1177
[LightGBM] [Info] Number of data points in the train set: 3870, number of used features: 16
LightGBM(SMOTE적용 전) -> 정확도 : 0.714, 정밀도 : 0.408, 재현율 : 0.170, f1-score : 0.240, AUC : 0.540
[LightGBM] [Info] Number of positive: 2886, number of negative: 2886
[LightGBM] [Info] Total Bins 3574
[LightGBM] [Info] Number of data po

* 오버샘플링(SMOTE)을 적용한 결과가 그렇지 않은 결과에 비해 전반적으로 우수하고, 특히 재현율과 정밀도에 있어서 큰 효과를 보임
* 4개의 알고리듬 중에는 재현율에 있어서는 Decision_tree가 정밀도나 f-score, AUC에 있어서는 XGBoost가 가장 우수한 성능을 보임.
* 종합적으로는 XGBoost가 가장 우수한 성능을 보인다고 판단 가능함

### (1-3-5) 독립변수 별 중도탈락예측 영향도 파악
* 중도탈락위험률 산정에 각 독립변수들이 어느 정도 영향을 미쳤는지에 대한 정량화된 수치로 중도탈락 예방을 위한 학생 관리 및 지원 프로그램에 대한 시사점(insight) 도출에 활용
* 성능측정 결과로 채택된 알고리듬 XGBoost를 기준으로 영향도가 큰 독립변수부터 출력

In [226]:
# 속성(feature) 별 영향도를 저장하는 데이터프레임 생성
feature_imp_xgboost = pd.DataFrame({'영향도(xgboost)' : gbm.feature_importances_}, index = feature_list[:-1])
feature_imp_dtree = pd.DataFrame({'영향도(dtree)' : decision_tree.feature_importances_}, index = feature_list[:-1])
feature_imp_rforest = pd.DataFrame({'영향도(rforest)' : random_forest.feature_importances_}, index = feature_list[:-1])
feature_imp_lgbm = pd.DataFrame({'영향도(lgbm)' : lgb.feature_importances_}, index = feature_list[:-1])

feature_imp = pd.merge(feature_imp_xgboost, feature_imp_dtree, left_index=True, right_index=True, how='inner')
feature_imp = pd.merge(feature_imp, feature_imp_rforest, left_index=True, right_index=True, how='inner')
feature_imp = pd.merge(feature_imp, feature_imp_lgbm, left_index=True, right_index=True, how='inner')
feature_imp.sort_values('영향도(xgboost)', ascending=False)

,영향도(xgboost),영향도(dtree),영향도(rforest),영향도(lgbm)
스펙업활동,0.125479,0.000000,0.053776,1670
지원활동,0.115701,0.000000,0.046427,2032
구직목표명확성,0.091237,0.610220,0.088265,4606
현재준비상태,0.089502,0.000000,0.066549,2938
구직기초지식,0.088291,0.019786,0.073367,4169
학점,0.063292,0.295157,0.094711,7204
자격증,0.059219,0.000000,0.046737,2825
구직의지,0.049782,0.000000,0.065026,5042
구직면접능력,0.048341,0.000000,0.063419,4024
정보탐색활동,0.043509,0.017146,0.065303,4593


* XGBoost를 기준으로 가장 영향도가 큰 독립변수는 스펙업활동으로 나타남 -> 자격증, 어학 등 스펙을 업그레이드 시킬 수 있는 비교과프로그램 수행 필요
* 독립변수 별 중도탈락 영향도는 분류·예측 알고리듬에 따라 다르게 나타날 수 있기 때문에 절대적으로 해석하기 보다 참고정보로 활용하는 것이 바람직함. 다만, 여러 알고리듬에서 공통적으로 영향도가 높게 나타난다면, 충분히 유의미할 수 있기 때문에 중요 영향변수로 간주하고 학생지원 프로그램 설계에 반영할 필요 있음

# 2. 예측대상 데이터 수집 및 예측모형 적용

## 2-1) 현(現) 졸업(예정)생 데이터 수집
* 데이터전처리 레이블인코딩 파트에서 NULL값을 가진 로우를 제거하고 범주형 특성을 레이블인코딩한 예측용 현 졸업(예정)생 데이터셋 활용
* 현 졸업생은 2021년 8월 또는 2022년 2월 기준 졸업생이고, 졸업예정자는 2022년 8월 또는 2023년 2월 졸업예정자임

## 2-2) 예측모형 적용
(1) 현(現) 졸업(예정)생 데이터셋을 대상으로 예측용 데이터셋 확보<br>
(2) 예측용 데이터셋을 예측모형에 적용하여 졸업(예정)생별 미취업위험률과 미취업위험순위 구하기<br>
(3) 현 졸업(예정)생 미취업위험 예측결과와 실제 미취업위험여부(정보공시취업기준) 비교<br>
(4) 현 졸업(예정)생 미취업위험 예측결과를 엑셀(excel)로 저장<br>

### (2-2-1) 현(現) 졸업(예정)생 데이터셋을 대상으로 예측용 데이터셋 확보
* 현(現) 졸업(예정)생의 예측대상 원천데이터셋이 확보되면 예측모형 개발과정에서 적용했던 데이터전처리 과정을 거쳐 예측모형 적용대상 데이터셋을 확보함

In [227]:
# 전처리된 현 졸업생 데이터셋을 저장하는 데이터프레임 확인
rawData2_e

,index,학생ID,소속학과,성별,취업준비유형,학점,구직기초지식,구직활동준비,정보탐색활동,스펙업활동,...,영어,기타어학,활동수준,현재준비상태,구직기술,구직서류작성능력,구직면접능력,구직의지,구직목표명확성,미취업여부
0,1,9971.0,0,0,3,2.76,44.80,24.13,30.27,30.42,...,39.2,26.15,28.92,19.33,52.88,46.12,58.41,51.99,38.26,0
1,2,10884.0,0,0,4,2.76,40.07,31.48,39.75,40.83,...,39.2,26.15,38.68,24.27,42.06,39.63,43.81,38.04,45.47,1
2,3,9646.0,0,0,0,2.43,47.16,43.67,47.33,46.04,...,39.2,26.15,47.77,39.58,51.08,49.37,51.11,72.15,65.29,0
3,4,11053.0,0,0,4,2.55,40.07,29.69,39.12,46.04,...,39.2,26.15,40.05,19.33,42.06,39.63,43.81,41.14,45.47,0
4,5,9206.0,0,0,4,2.75,35.34,27.31,34.69,40.83,...,39.2,26.15,35.28,19.33,42.66,45.04,38.94,47.34,43.67,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1866,2066,9482.0,34,1,3,3.61,47.16,34.54,39.12,51.25,...,39.2,26.15,44.82,24.27,51.68,50.45,51.11,62.84,56.28,0
1867,2067,10409.0,34,1,4,3.24,44.80,28.98,30.27,35.63,...,39.2,26.15,33.69,24.27,43.86,46.12,40.16,58.19,40.06,0
1868,2068,9096.0,34,1,3,3.68,56.62,47.15,73.88,66.88,...,39.2,26.15,70.03,24.27,58.89,55.86,59.62,61.29,43.67,1
1869,2069,11277.0,34,1,1,4.28,56.62,47.40,56.18,51.25,...,39.2,26.15,55.72,39.09,60.09,59.10,58.41,72.15,65.29,0


In [228]:
# 전처리된 현 졸업예정자 데이터셋을 저장하는 데이터프레임 확인
rawData3_e

,index,학생ID,소속학과,성별,취업준비유형,학점,구직기초지식,구직활동준비,정보탐색활동,스펙업활동,...,영어,기타어학,활동수준,현재준비상태,구직기술,구직서류작성능력,구직면접능력,구직의지,구직목표명확성,미취업여부
0,5,11844.0,0,0,4,3.00,37.71,30.00,35.95,35.63,...,39.2,26.15,35.73,24.27,43.86,51.53,34.08,38.04,40.06,2
1,6,13962.0,0,0,0,1.67,44.80,24.13,30.27,30.42,...,39.2,26.15,28.92,19.33,25.23,24.49,26.78,72.15,67.09,2
2,7,11839.0,0,1,3,2.65,61.35,37.23,37.85,35.63,...,39.2,26.15,42.77,31.68,61.30,60.18,59.62,73.70,40.06,2
3,8,11804.0,0,0,3,2.85,56.62,34.53,45.44,51.25,...,39.2,26.15,42.32,26.74,51.68,41.80,60.84,56.64,52.68,2
4,9,11599.0,0,0,4,3.28,37.71,26.16,30.27,35.63,...,39.2,26.15,30.51,21.80,43.86,42.88,43.81,41.14,41.87,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1681,2103,12044.0,34,1,4,3.93,44.80,32.60,34.69,46.04,...,39.2,26.15,38.46,26.74,45.06,46.12,42.59,38.04,38.26,2
1682,2104,11899.0,34,0,4,3.71,35.34,32.36,35.32,40.83,...,39.2,26.15,35.51,29.21,43.86,41.80,45.02,55.09,41.87,2
1683,2105,11775.0,34,0,3,3.66,58.98,39.26,46.70,40.83,...,39.2,26.15,44.36,34.15,45.67,46.12,43.81,41.14,45.47,2
1684,2106,12029.0,34,1,1,4.36,58.98,49.18,49.86,51.25,...,39.2,26.15,51.86,46.49,70.31,63.43,74.22,67.49,59.89,2


### (2-2-2) 예측용 데이터셋을 예측모형에 적용하여 졸업(예정)생별 미취업위험률과 미취업위험순위 구하기
* 졸업(예정)생 개인별 미취업위험예측 결과로 다음 항목을 도출할 수 있음
 * 졸업(예정)별 미취업위험위험률
   * 졸업(예정)별로 미취업위험 정도를 나타내는 백분위값
   * 예측모형의 함수를 활용하여 구할 수 있음
 * 졸업(예정)별 미취업위험순위
   * 학과/반별 미취업위험순위 -> 순위가 높을수록 위험정도가 높으며, 반별 지도교수가 집중 지도 및 관리 졸업(예정)생을 선별하는데 활용
   * 각 졸업(예정)생들의 학생ID로 학번을 파악하고, 지도반별 졸업(예정)들의 중도탈락위험률을 내림차순 정렬하여 구할 수 있음

In [229]:
# 분석대상 특성리스트 (독립변수만)
feature_ind_list = feature_list[:-1]

# 분석대상 특성으로 구성된 예측용 데이터셋
expData2 = rawData2_e[feature_ind_list]  ## 졸업생 데이터셋
expData3 = rawData3_e[feature_ind_list]  ## 졸업예정자 데이터셋

# 결측치 제거
expData2.dropna(inplace=True)
expData2.reset_index(inplace=True, drop=True)  # 인덱스 재설정

expData3.dropna(inplace=True)
expData3.reset_index(inplace=True, drop=True)  # 인덱스 재설정

In [230]:
# 예측용 졸업생 데이터셋
expData2

,소속학과,취업준비유형,학점,구직기초지식,구직활동준비,정보탐색활동,스펙업활동,지원활동,자격증,활동수준,현재준비상태,구직기술,구직서류작성능력,구직면접능력,구직의지,구직목표명확성
0,0,3,2.76,44.80,24.13,30.27,30.42,31.37,39.00,28.92,19.33,52.88,46.12,58.41,51.99,38.26
1,0,4,2.76,40.07,31.48,39.75,40.83,38.78,52.33,38.68,24.27,42.06,39.63,43.81,38.04,45.47
2,0,0,2.43,47.16,43.67,47.33,46.04,49.89,39.00,47.77,39.58,51.08,49.37,51.11,72.15,65.29
3,0,4,2.55,40.07,29.69,39.12,46.04,38.78,39.00,40.05,19.33,42.06,39.63,43.81,41.14,45.47
4,0,4,2.75,35.34,27.31,34.69,40.83,35.07,39.00,35.28,19.33,42.66,45.04,38.94,47.34,43.67
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1866,34,3,3.61,47.16,34.54,39.12,51.25,46.19,39.00,44.82,24.27,51.68,50.45,51.11,62.84,56.28
1867,34,4,3.24,44.80,28.98,30.27,35.63,38.78,39.00,33.69,24.27,43.86,46.12,40.16,58.19,40.06
1868,34,3,3.68,56.62,47.15,73.88,66.88,64.70,39.00,70.03,24.27,58.89,55.86,59.62,61.29,43.67
1869,34,1,4.28,56.62,47.40,56.18,51.25,57.30,52.33,55.72,39.09,60.09,59.10,58.41,72.15,65.29


In [231]:
# 예측용 졸업예정자 데이터셋
expData3

,소속학과,취업준비유형,학점,구직기초지식,구직활동준비,정보탐색활동,스펙업활동,지원활동,자격증,활동수준,현재준비상태,구직기술,구직서류작성능력,구직면접능력,구직의지,구직목표명확성
0,0,4,3.00,37.71,30.00,35.95,35.63,38.78,39.00,35.73,24.27,43.86,51.53,34.08,38.04,40.06
1,0,0,1.67,44.80,24.13,30.27,30.42,31.37,39.00,28.92,19.33,25.23,24.49,26.78,72.15,67.09
2,0,3,2.65,61.35,37.23,37.85,35.63,53.59,72.33,42.77,31.68,61.30,60.18,59.62,73.70,40.06
3,0,3,2.85,56.62,34.53,45.44,51.25,35.07,45.67,42.32,26.74,51.68,41.80,60.84,56.64,52.68
4,0,4,3.28,37.71,26.16,30.27,35.63,31.37,39.00,30.51,21.80,43.86,42.88,43.81,41.14,41.87
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1681,34,4,3.93,44.80,32.60,34.69,46.04,38.78,39.00,38.46,26.74,45.06,46.12,42.59,38.04,38.26
1682,34,4,3.71,35.34,32.36,35.32,40.83,35.07,45.67,35.51,29.21,43.86,41.80,45.02,55.09,41.87
1683,34,3,3.66,58.98,39.26,46.70,40.83,46.19,65.67,44.36,34.15,45.67,46.12,43.81,41.14,45.47
1684,34,1,4.36,58.98,49.18,49.86,51.25,53.59,85.67,51.86,46.49,70.31,63.43,74.22,67.49,59.89


In [232]:
# predict() : 모형변수의 함수로서 졸업생 각각에 대한 미취업위험 예측
y_pred_dtree_2 = decision_tree.predict(expData2)    # 의사결정트리
y_pred_rforest_2 = random_forest.predict(expData2)  # 랜덤포레스트
y_pred_xgboost_2 = gbm.predict(expData2)            # XGBoost
y_pred_lgbm_2 = lgb.predict(expData2)               # LightGBM

# predict() : 모형변수의 함수로서 졸업예정자 각각에 대한 미취업위험 예측
y_pred_dtree_3 = decision_tree.predict(expData3)    # 의사결정트리
y_pred_rforest_3 = random_forest.predict(expData3)  # 랜덤포레스트
y_pred_xgboost_3 = gbm.predict(expData3)            # XGBoost
y_pred_lgbm_3 = lgb.predict(expData3)               # LightGBM

# predict_proba() : 모형변수의 함수로서 졸업생 각각에 대한 미취업위험 예측
y_pred_prob_dtree_2 = decision_tree.predict_proba(expData2)     # 의사결정트리
y_pred_prob_rforest_2 = random_forest.predict_proba(expData2)   # 랜덤포레스트
y_pred_prob_xgboost_2 = gbm.predict_proba(expData2)             # GBoost
y_pred_prob_lgbm_2 = lgb.predict_proba(expData2)                # LightGBM

# predict_proba() : 모형변수의 함수로서 졸업예정자 각각에 대한 미취업위험 예측
y_pred_prob_dtree_3 = decision_tree.predict_proba(expData3)     # 의사결정트리
y_pred_prob_rforest_3 = random_forest.predict_proba(expData3)   # 랜덤포레스트
y_pred_prob_xgboost_3 = gbm.predict_proba(expData3)             # GBoost
y_pred_prob_lgbm_3 = lgb.predict_proba(expData3)                # LightGBM

In [233]:
# 졸업생 미취업위험예측결과를 데이터프레임에 결합
expData2.loc[:, '미취업여부예측(dtree)'] = y_pred_dtree_2
expData2.loc[:, '미취업위험률(dtree)'] = y_pred_prob_dtree_2[:,1]

expData2.loc[:, '미취업여부예측(rforest)'] = y_pred_rforest_2
expData2.loc[:, '미취업위험률(rforest)'] = y_pred_prob_rforest_2[:,1]

expData2.loc[:, '미취업여부예측(xgboost)'] = y_pred_xgboost_2
expData2.loc[:, '미취업위험률(xgboost)'] = y_pred_prob_xgboost_2[:,1]

expData2.loc[:, '미취업여부예측(lgbm)'] = y_pred_lgbm_2
expData2.loc[:, '미취업위험률(lgbm)'] = y_pred_prob_lgbm_2[:,1]

expData2

,소속학과,취업준비유형,학점,구직기초지식,구직활동준비,정보탐색활동,스펙업활동,지원활동,자격증,활동수준,...,구직의지,구직목표명확성,미취업여부예측(dtree),미취업위험률(dtree),미취업여부예측(rforest),미취업위험률(rforest),미취업여부예측(xgboost),미취업위험률(xgboost),미취업여부예측(lgbm),미취업위험률(lgbm)
0,0,3,2.76,44.80,24.13,30.27,30.42,31.37,39.00,28.92,...,51.99,38.26,1,0.668440,0,0.40,1,0.530279,0,7.923163e-04
1,0,4,2.76,40.07,31.48,39.75,40.83,38.78,52.33,38.68,...,38.04,45.47,0,0.359833,0,0.45,0,0.259645,0,2.993167e-03
2,0,0,2.43,47.16,43.67,47.33,46.04,49.89,39.00,47.77,...,72.15,65.29,1,0.516274,0,0.40,0,0.314171,0,1.272007e-02
3,0,4,2.55,40.07,29.69,39.12,46.04,38.78,39.00,40.05,...,41.14,45.47,0,0.359833,0,0.40,0,0.182021,0,4.407411e-06
4,0,4,2.75,35.34,27.31,34.69,40.83,35.07,39.00,35.28,...,47.34,43.67,0,0.333333,0,0.50,1,0.552538,0,1.464070e-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1866,34,3,3.61,47.16,34.54,39.12,51.25,46.19,39.00,44.82,...,62.84,56.28,0,0.410867,0,0.20,0,0.084011,0,1.070834e-03
1867,34,4,3.24,44.80,28.98,30.27,35.63,38.78,39.00,33.69,...,58.19,40.06,1,0.668440,0,0.15,0,0.085977,0,2.581424e-06
1868,34,3,3.68,56.62,47.15,73.88,66.88,64.70,39.00,70.03,...,61.29,43.67,0,0.410867,0,0.30,0,0.045346,0,8.817790e-07
1869,34,1,4.28,56.62,47.40,56.18,51.25,57.30,52.33,55.72,...,72.15,65.29,0,0.237895,0,0.15,0,0.170668,0,2.207574e-06


In [234]:
# 졸업예정자 미취업위험예측결과를 데이터프레임에 결합
expData3.loc[:, '미취업여부예측(dtree)'] = y_pred_dtree_3
expData3.loc[:, '미취업위험률(dtree)'] = y_pred_prob_dtree_3[:,1]

expData3.loc[:, '미취업여부예측(rforest)'] = y_pred_rforest_3
expData3.loc[:, '미취업위험률(rforest)'] = y_pred_prob_rforest_3[:,1]

expData3.loc[:, '미취업여부예측(xgboost)'] = y_pred_xgboost_3
expData3.loc[:, '미취업위험률(xgboost)'] = y_pred_prob_xgboost_3[:,1]

expData3.loc[:, '미취업여부예측(lgbm)'] = y_pred_lgbm_3
expData3.loc[:, '미취업위험률(lgbm)'] = y_pred_prob_lgbm_3[:,1]

expData3

,소속학과,취업준비유형,학점,구직기초지식,구직활동준비,정보탐색활동,스펙업활동,지원활동,자격증,활동수준,...,구직의지,구직목표명확성,미취업여부예측(dtree),미취업위험률(dtree),미취업여부예측(rforest),미취업위험률(rforest),미취업여부예측(xgboost),미취업위험률(xgboost),미취업여부예측(lgbm),미취업위험률(lgbm)
0,0,4,3.00,37.71,30.00,35.95,35.63,38.78,39.00,35.73,...,38.04,40.06,1,0.668440,0,0.35,0,0.337815,0,9.874144e-03
1,0,0,1.67,44.80,24.13,30.27,30.42,31.37,39.00,28.92,...,72.15,67.09,1,0.516274,0,0.15,0,0.091195,0,7.722698e-08
2,0,3,2.65,61.35,37.23,37.85,35.63,53.59,72.33,42.77,...,73.70,40.06,1,0.668440,0,0.25,0,0.105546,0,4.898258e-05
3,0,3,2.85,56.62,34.53,45.44,51.25,35.07,45.67,42.32,...,56.64,52.68,1,0.516274,1,0.55,0,0.373396,0,1.953592e-02
4,0,4,3.28,37.71,26.16,30.27,35.63,31.37,39.00,30.51,...,41.14,41.87,1,0.668440,0,0.45,0,0.063771,0,2.034935e-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1681,34,4,3.93,44.80,32.60,34.69,46.04,38.78,39.00,38.46,...,38.04,38.26,0,0.377863,0,0.15,0,0.019119,0,1.319581e-07
1682,34,4,3.71,35.34,32.36,35.32,40.83,35.07,45.67,35.51,...,55.09,41.87,0,0.377863,0,0.20,0,0.069297,0,1.089139e-06
1683,34,3,3.66,58.98,39.26,46.70,40.83,46.19,65.67,44.36,...,41.14,45.47,0,0.410867,0,0.20,0,0.030527,0,2.220256e-07
1684,34,1,4.36,58.98,49.18,49.86,51.25,53.59,85.67,51.86,...,67.49,59.89,0,0.237895,0,0.25,0,0.125873,0,2.495541e-06


### (2-2-3) 현 졸업(예정)생 미취업위험 예측결과와 실제 미취업여부
* 현 졸업생 데이터셋에 정보공시취업데이터 기준 미취업여부가 존재하므로 예측결과와 비교
* 졸업예정자의 경우는 정보공시취업데이터가 존재하지 않으므로 예측결과와 비교 불가

In [235]:
# 예측용 졸업생 데이터셋에 학생ID와 실제 미취업여부(정보공시기준) 컬럼 추가
rawData2_e.dropna(inplace=True)
rawData2_e.reset_index(inplace=True)  # 인덱스 재설정
expData2['학생ID'] = rawData2_e['학생ID']
expData2['미취업여부'] = rawData2_e['미취업여부']

# 예측용 졸업예정자 데이터셋에 학생ID 컬럼 추가 (실제 미취업여부 데이터는 아직 없음)
rawData3_e.dropna(inplace=True)
rawData3_e.reset_index(inplace=True)  # 인덱스 재설정
expData3['학생ID'] = rawData3_e['학생ID']

In [236]:
# 예측결과와 실제결과가 일치하는 건수를 알고리듬별로 저장하는 변수 초기화 (졸업생 데이터셋 대상, 졸업예정자는 불가)
cnt_df = 0
cnt_rf = 0
cnt_xg = 0
cnt_lg = 0

# 일치여부 확인
for i in range(0, len(expData2)) :
    if expData2['미취업여부'].values[i] == expData2['미취업여부예측(dtree)'].values[i] :
        cnt_df += 1

    if expData2['미취업여부'].values[i] == expData2['미취업여부예측(rforest)'].values[i] :
        cnt_rf += 1

    if expData2['미취업여부'].values[i] == expData2['미취업여부예측(xgboost)'].values[i] :
        cnt_xg += 1

    if expData2['미취업여부'].values[i] == expData2['미취업여부예측(lgbm)'].values[i] :
        cnt_lg += 1

# 결과 출력
print("Decision tree : 적중개수 %d, 적중률 %.2f" % (cnt_df, cnt_df / len(expData2) * 100))
print("Random Forest : 적중개수 %d, 적중률 %.2f" % (cnt_rf, cnt_rf / len(expData2) * 100))
print("XGBoost : 적중개수 %d, 적중률 %.2f" % (cnt_xg, cnt_xg / len(expData2) * 100))
print("Light GBM : 적중개수 %d, 적중률 %.2f" % (cnt_lg, cnt_lg / len(expData2) * 100))

Decision tree : 적중개수 1133, 적중률 60.56
Random Forest : 적중개수 1360, 적중률 72.69
XGBoost : 적중개수 1398, 적중률 74.72
Light GBM : 적중개수 1404, 적중률 75.04


* 분석차원(독립변수)를 줄였을 때와 그렇지 않았을 때 모두 XGBoost와 Light GBM의 적중률이 상대적으로 높았음

In [237]:
# 현 졸업생 원천데이터셋에 예측결과 결합
result_df2 = pd.merge(rawData2_org, expData2[['학생ID','미취업여부예측(dtree)','미취업위험률(dtree)'
                                             ,'미취업여부예측(rforest)','미취업위험률(rforest)'
                                             ,'미취업여부예측(xgboost)','미취업위험률(xgboost)'
                                             ,'미취업여부예측(lgbm)','미취업위험률(lgbm)']], how='left', on='학생ID')
result_df2

,학생ID,성별,소속학과,지도교수,반,학점,취업준비유형,구직기초지식,구직활동준비,정보탐색활동,...,미취업여부,구분,미취업여부예측(dtree),미취업위험률(dtree),미취업여부예측(rforest),미취업위험률(rforest),미취업여부예측(xgboost),미취업위험률(xgboost),미취업여부예측(lgbm),미취업위험률(lgbm)
0,10098,여자,학과01,김진동,B,2.74,NaN,NaN,NaN,NaN,...,1,예측용(졸업생),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,9971,남자,학과01,김진동,B,2.76,중간집단,44.80,24.13,30.27,...,0,예측용(졸업생),1.0,0.668440,0.0,0.40,1.0,0.530279,0.0,7.923163e-04
2,10884,남자,학과01,김진동,B,2.76,취업적신호,40.07,31.48,39.75,...,1,예측용(졸업생),0.0,0.359833,0.0,0.45,0.0,0.259645,0.0,2.993167e-03
3,9646,남자,학과01,김진동,B,2.43,마음만 취준생,47.16,43.67,47.33,...,0,예측용(졸업생),1.0,0.516274,0.0,0.40,0.0,0.314171,0.0,1.272007e-02
4,11053,남자,학과01,김진동,B,2.55,취업적신호,40.07,29.69,39.12,...,0,예측용(졸업생),0.0,0.359833,0.0,0.40,0.0,0.182021,0.0,4.407411e-06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2066,9482,여자,학과35,이상준,B,3.61,중간집단,47.16,34.54,39.12,...,0,예측용(졸업생),0.0,0.410867,0.0,0.20,0.0,0.084011,0.0,1.070834e-03
2067,10409,여자,학과35,이상준,B,3.24,취업적신호,44.80,28.98,30.27,...,0,예측용(졸업생),1.0,0.668440,0.0,0.15,0.0,0.085977,0.0,2.581424e-06
2068,9096,여자,학과35,이상준,B,3.68,중간집단,56.62,47.15,73.88,...,1,예측용(졸업생),0.0,0.410867,0.0,0.30,0.0,0.045346,0.0,8.817790e-07
2069,11277,여자,학과35,이상준,B,4.28,준비된 취준생,56.62,47.40,56.18,...,0,예측용(졸업생),0.0,0.237895,0.0,0.15,0.0,0.170668,0.0,2.207574e-06


In [238]:
# 현 졸업생 원천데이터셋에 예측결과 결합
result_df3 = pd.merge(rawData3_org, expData3[['학생ID','미취업여부예측(dtree)','미취업위험률(dtree)'
                                             ,'미취업여부예측(rforest)','미취업위험률(rforest)'
                                             ,'미취업여부예측(xgboost)','미취업위험률(xgboost)'
                                             ,'미취업여부예측(lgbm)','미취업위험률(lgbm)']], how='left', on='학생ID')
result_df3

,학생ID,성별,소속학과,지도교수,학점,취업준비유형,구직기초지식,구직활동준비,정보탐색활동,스펙업활동,...,미취업여부,구분,미취업여부예측(dtree),미취업위험률(dtree),미취업여부예측(rforest),미취업위험률(rforest),미취업여부예측(xgboost),미취업위험률(xgboost),미취업여부예측(lgbm),미취업위험률(lgbm)
0,13961.0,여자,학과01,김진동,2.77,NaN,NaN,NaN,NaN,NaN,...,NaN,예측용(졸업예정자),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,11470.0,남자,학과01,김진동,2.99,NaN,NaN,NaN,NaN,NaN,...,NaN,예측용(졸업예정자),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,13626.0,남자,학과01,김진동,3.24,NaN,NaN,NaN,NaN,NaN,...,NaN,예측용(졸업예정자),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,11480.0,남자,학과01,김진동,3.53,NaN,NaN,NaN,NaN,NaN,...,NaN,예측용(졸업예정자),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,13628.0,여자,학과01,김진동,3.88,NaN,NaN,NaN,NaN,NaN,...,NaN,예측용(졸업예정자),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2103,12044.0,여자,학과35,이상준,3.93,취업적신호,44.80,32.60,34.69,46.04,...,NaN,예측용(졸업예정자),0.0,0.377863,0.0,0.15,0.0,0.019119,0.0,1.319581e-07
2104,11899.0,남자,학과35,이상준,3.71,취업적신호,35.34,32.36,35.32,40.83,...,NaN,예측용(졸업예정자),0.0,0.377863,0.0,0.20,0.0,0.069297,0.0,1.089139e-06
2105,11775.0,남자,학과35,이상준,3.66,중간집단,58.98,39.26,46.70,40.83,...,NaN,예측용(졸업예정자),0.0,0.410867,0.0,0.20,0.0,0.030527,0.0,2.220256e-07
2106,12029.0,여자,학과35,이상준,4.36,준비된 취준생,58.98,49.18,49.86,51.25,...,NaN,예측용(졸업예정자),0.0,0.237895,0.0,0.25,0.0,0.125873,0.0,2.495541e-06


### (2-2-4) 현 졸업(예정)생 중도탈락위험 예측결과를 엑셀(excel)로 저장

In [239]:
# XlsxWriter 엔진으로 Pandas writer 객체 만들기
if feature_set_shrink == 1:  # 차원축소한 경우
    writer = pd.ExcelWriter('졸업(예정)생별_미취업위험예측(차원축소)결과.xlsx')
else: # 차원축소하지 않은 경우
    writer = pd.ExcelWriter('졸업(예정)생별_미취업위험예측결과.xlsx')

# 데이터프레임 내용을 엑셀에 저장
result_df2.to_excel(writer, sheet_name='졸업생')
result_df3.to_excel(writer, sheet_name='졸업예정자')

# Pandas writer 객체 닫기
writer.close()